In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
from hard_data import generate_data_head_shoulder, generate_sample_df_with_pattern
from analysis import detect_and_rename

In [2]:
def plot_simple_df(candle_data: pd.DataFrame, ticker: str,  count: int, pad: int = 2):
    if pad < 0:
        pad = 0
    fname = "images/syn/" + 'hs_' + ticker + '_' + str(count) + ".png"
    
    candle_data.index = pd.DatetimeIndex(candle_data['Datetime'])
    candle_data.drop(columns=['Datetime'], inplace=True)

    mpf.plot(candle_data, type='candle', axisoff=True, style = 'classic', savefig=dict(fname=fname, dpi=60))  

In [5]:
ticker = 'AAPL'
data = pd.read_csv('data/' + ticker + '_data.csv', index_col=None, header=0)
# Place the datetime in a temp dataset
temp = data['Datetime']
data.rename(columns={"open": "Open", "close": "Close", "high": "High",
                     "low": "Low", 'volume': 'Volume'}, inplace=True)
# Apply numeric and log to the data
data = data.apply(pd.to_numeric, errors='coerce')
data = np.log(data)
# Add the datetime we saved earlier back in
data['Datetime'] = temp
data.drop(columns=['adj_close', 'adj_high', 'adj_low'], inplace=True)

In [6]:
data = detect_and_rename(data, 'wavelet', 3)

In [8]:
data.head()

,Datetime,Open,High,Low,Close,Volume,High_smooth,Low_smooth,high_roll_max,low_roll_min,head_shoulder_pattern_wavelet
0,2022-04-19 09:30:00,5.106067,5.115055,5.099318,5.113643,16.590533,5.092125,5.077974,NaN,NaN,NaN
1,2022-04-19 10:30:00,5.113613,5.120625,5.113312,5.119370,15.982426,5.094615,5.088723,NaN,NaN,NaN
2,2022-04-19 11:30:00,5.119400,5.120685,5.114875,5.114995,15.589646,5.094134,5.086783,5.094615,5.077974,NaN
3,2022-04-19 12:30:00,5.114935,5.116076,5.112710,5.112751,15.296711,5.092605,5.086783,5.094615,5.086783,NaN
4,2022-04-19 13:30:00,5.112771,5.115896,5.111114,5.114214,15.573481,5.090411,5.084909,5.094134,5.084909,Head and Shoulder


In [9]:
data1 = data[data['head_shoulder_pattern_wavelet'].notnull()]
len(data1.index)

195

In [10]:
data1

,Datetime,Open,High,Low,Close,Volume,High_smooth,Low_smooth,high_roll_max,low_roll_min,head_shoulder_pattern_wavelet
4,2022-04-19 13:30:00,5.112771,5.115896,5.111114,5.114214,15.573481,5.090411,5.084909,5.094134,5.084909,Head and Shoulder
9,2022-04-20 11:30:00,5.116796,5.119699,5.112590,5.114900,15.691351,5.089663,5.083472,5.096583,5.083472,Head and Shoulder
16,2022-04-21 11:30:00,5.142745,5.143037,5.134032,5.134238,16.219502,5.113258,5.104092,5.113258,5.088875,Inverse Head and Shoulder
20,2022-04-21 15:30:00,5.116196,5.118802,5.111988,5.114094,16.004316,5.097642,5.087607,5.105226,5.087607,Head and Shoulder
25,2022-04-22 13:30:00,5.100293,5.101511,5.095650,5.096664,15.788977,5.082245,5.076542,5.095164,5.076542,Head and Shoulder
...,...,...,...,...,...,...,...,...,...,...,...
3335,2024-03-14 10:30:00,5.157157,5.160823,5.155024,5.155255,15.568776,5.174321,5.168131,5.174321,5.160642,Inverse Head and Shoulder
3349,2024-03-18 10:30:00,5.178238,5.180153,5.173661,5.175132,15.975040,5.185064,5.178513,5.185064,5.155356,Inverse Head and Shoulder
3405,2024-03-28 10:30:00,5.142832,5.142891,5.138794,5.140698,15.882775,5.167421,5.162463,5.171579,5.162463,Head and Shoulder
3473,2024-04-11 15:30:00,5.163299,5.167411,5.162612,5.165014,16.524150,5.182972,5.182183,5.182972,5.169019,Inverse Head and Shoulder


In [11]:
loc = 0
for i in data1.index:
    if i + 15 < len(data) and loc < i and i > 15:
        loc += 40
        plot_simple_df(data.loc[i - 15 : i+15], ticker, loc, 2)

In [2]:
df = generate_data_head_shoulder(1)
df = detect_and_rename(df, 'wavelet', 3)
df

C:\Users\Nick\Documents\SchoolStuff\spring2024\machineLearning\TradingPatternScanner-main\tradingpatterns\tradingpatterns_tech.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Head and Shoulder' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask_head_shoulder, 'head_shoulder_pattern'] = 'Head and Shoulder'


,Open,High,Low,Close,Volume,High_smooth,Low_smooth,high_roll_max,low_roll_min,head_shoulder_pattern_wavelet
Datetime,,,,,,,,,,
2024-04-22 17:14:55.094336,126,134,126,132,1861,141.381008,145.580152,NaN,NaN,NaN
2024-04-23 17:14:55.094336,187,189,179,188,1277,174.320807,163.660924,NaN,NaN,NaN
2024-04-24 17:14:55.094336,193,199,191,197,1753,184.820807,167.388664,184.820807,145.580152,NaN
2024-04-25 17:14:55.094336,176,123,170,122,1741,130.881008,167.388664,184.820807,163.660924,NaN
2024-04-26 17:14:55.094336,128,138,129,130,1164,151.494920,128.651649,184.820807,128.651649,NaN
2024-04-27 17:14:55.094336,119,161,127,159,1682,152.434718,128.651649,152.434718,128.651649,NaN
2024-04-28 17:14:55.094336,102,120,107,110,1598,134.083366,128.651649,152.434718,128.651649,NaN
2024-04-29 17:14:55.094336,146,123,140,122,1681,134.083366,128.651649,152.434718,128.651649,NaN
2024-04-30 17:14:55.094336,172,176,169,175,1417,175.839819,160.482414,175.839819,128.651649,NaN


In [8]:
plot_simple_df(df, 0, 2)